# Check the status of uploaded experiments

## Prerequisites

In [31]:
import datetime
import importlib
import re
from tqdm import tqdm

import plotnine as p9

import pandas as pd

from utils import add_src_to_sys_path

add_src_to_sys_path()

from common import wandb_utils, nest

wandb_utils = importlib.reload(wandb_utils)

## Load the experiments

In [32]:
wandb_api = wandb_utils.get_wandb_api()

In [33]:
# Filter runs by tags and exclude runs with job_type "attn"
runs = wandb_api.runs(
    f"{wandb_utils.get_entity_name()}/{wandb_utils.get_project_name()}",
    filters={
        "tags": {
            "$in": ["scratchpad_f", "classic", "sanity_check"]
        },
        "jobType": {
            "$nin": ["attn_analysis2", "attn_analysis", "attn_analysis_aggr", "attn_analysis_kl"]
        }
    }
)

runs = list(runs)
len(runs)

1682

In [34]:
manually_checked_to_be_complete = {
    "sw_16381da468a2f32100a5a050ca713ba7__146317",
    "sw_844b3f59c315724dd2b6d10f911423f4__256788",
    "sw_1cf333ff3ac676ce168f7ed166e63dfc__234054",
    "sw_2b0f996045d25630bb36a80512ad82fa__146317",
    "sw_be00a87994aa75aef0e45b7f7714ac53__146317",
    "sw_2d57a9b60c34a380d3f1eeeb8b422815__146317",
    "sw_c28163735f1365dbaf814749d4a26006__146317",
}

In [35]:
def get_model_name(group_str: str) -> str:
    pe_names = ["pe_none", "pe_t5", "pe_abs_sin", "pe_alibi", "pe_rotary", "pe_newRot"]
    for pe in pe_names:
        if pe in group_str:
            return pe
    raise ValueError("Invalid pe")

def get_launcher(run) -> str:
    if run.job_type == "agent":
        return run.id

    launcher_tag = [t for t in run.tags if t.startswith("launched_by_")][0]
    return launcher_tag.split("launched_by_")[1]


def check_runs_generated_by_launcher_id(launcher_id: str) -> None:
    l_runs = wandb_api.runs(
        f"{wandb_utils.get_entity_name()}/{wandb_utils.get_project_name()}",
        filters={
            "tags": {
                "$in": ["launched_by_" + launcher_id]
            }
        }
    )

    l_runs = list(l_runs)

    for run in l_runs:
        if run.job_type == "agent":
            continue
        print(run.id, run.state, run.job_type, run.group, run.tags)
        is_complete = is_run_complete(run)
        print(is_complete)
        print()


def is_run_complete(run) -> bool:
    if run.id in manually_checked_to_be_complete:
        return True

    # Assert training is done
    try:
        max_steps = run.config["trainer"]["max_steps"]
    except Exception as e:
        print("Could not get max_steps")
        return False

    h = run.history(samples=500, keys=["train/loss"], x_axis="train/global_step")
    if h is None or len(h) == 0:
        print(run.id, run.group)
        print("Not enough train summary")
        return False
    last_step = max(h["train/global_step"].tolist())

    if abs(last_step - max_steps) > 0.01 * max_steps:
        print("Not enough train steps")
        return False

    # Assert validation is done
    if (
        "pred/test_acc_overall" not in run.summary
        or run.summary["pred/test_acc_overall"] is None
    ):
        # h = run.history(keys=["pred/test_acc_overall"])
        # if h is None or len(h) == 0:
        #     return False
        print(run.id, run.group)
        print("no eval results")
        return False

    return True

df_data = []

scratchpad_config_pattern = re.compile(r"(.)*_scratchpad(.)+_ufs__(i._c._o._v._r.)_.*___.*")
dataset_name_pattern = re.compile(f".*___data-(.+)-(.+)")

for run in tqdm(runs):
    group = run.group

    if "attn" in run.job_type:
        continue

    if "data-sum" in run.group:
        continue

    scratchpad_config = "no_scratchpad"
    result = scratchpad_config_pattern.search(group)
    if result:
        scratchpad_config = result.group(3)

    result = dataset_name_pattern.search(group)
    dataset_name = result.group(1)
    dataset_split = result.group(2)
    dataset_comb = f"{dataset_name}--{dataset_split}"

    model_name = get_model_name(group)

    if run.job_type == "agent":
        is_complete = True
    else:
        is_complete = is_run_complete(run)

    df_data.append({
        "run_group": group,
        "job_type": run.job_type,
        "launcher_id": get_launcher(run),
        "ds_name": dataset_name,
        "ds_split": dataset_split,
        "ds": dataset_comb,
        "scratchpad_config": scratchpad_config,
        "model": model_name,
        "is_complete": is_complete,
        "state": run.state,
        "id": run.id,
        "gr_url": f"https://wandb.ai/kzmnjd/len_gen/groups/{group}",
        "run_url": run.url,
        "created_at": run.created_at,
        "host": run.host,
    })

df = pd.DataFrame.from_records(df_data)
# df

  0%|          | 4/1682 [00:00<03:40,  7.60it/s]

Not enough train steps
Not enough train steps


  0%|          | 6/1682 [00:00<03:36,  7.75it/s]

Not enough train steps


  0%|          | 8/1682 [00:01<03:34,  7.80it/s]

...

 38%|███▊      | 636/1682 [00:48<00:13, 76.74it/s]

Not enough train steps


 66%|██████▌   | 1113/1682 [01:26<01:13,  7.79it/s]

Not enough train steps


100%|██████████| 1682/1682 [02:17<00:00, 12.19it/s] 


In [36]:
def get_compute_cluster(host: str):
    if "cedar" in host:
        return "cc_cedar"
    elif "narval" in host:
        return "cc_narval"
    elif host.startswith("cn-"):
        return "mila"
    else:
        return "unknown"


def get_grouped_df(gdf):
    seed_runs = gdf[gdf.job_type == "best_run_seed_exp"]
    completed = len(seed_runs[seed_runs.is_complete == True])
    is_running = "running" in gdf.state.unique().tolist()
    group_url = gdf.gr_url.tolist()[0]
    host = seed_runs.host.tolist()[0] if len(seed_runs) > 0 else "unknown"
    launcher_ids = gdf[gdf.job_type == "agent"][["launcher_id", "created_at"]]
    launcher_ids = [tuple(x) for x in launcher_ids.values]
    launcher_ids.sort(key=lambda x: datetime.datetime.fromisoformat(x[1]), reverse=True)
    launcher_id = launcher_ids[0][0]

    return pd.DataFrame.from_records([{
        "num_completed": completed,
        "num_seed_runs": len(seed_runs),
        "is_running": is_running,
        "is_done": completed >= 3,
        "launcher_id": launcher_id,
        "compute_cluster": get_compute_cluster(host),
        "group_url": group_url
    }])


In [37]:
xdf = df.groupby(["ds", "model", "scratchpad_config"]).apply(get_grouped_df)
xdf

num_completed  \
ds                    model      scratchpad_config                    
pcfg--md_productivity pe_abs_sin no_scratchpad     0              3   
                      pe_alibi   no_scratchpad     0              3   
                      pe_none    no_scratchpad     0              3   
                      pe_rotary  no_scratchpad     0              3   
                      pe_t5      no_scratchpad     0              3   
...                                                             ...   
scan--mdlen_tr25_ts48 pe_abs_sin no_scratchpad     0              3   
                      pe_alibi   no_scratchpad     0              3   
                      pe_none    no_scratchpad     0              3   
                      pe_rotary  no_scratchpad     0              3   
                      pe_t5      no_scratchpad     0              3   

                                                      num_seed_runs  \
ds                    model      scratchpad_config                    
pcfg--md_productivity pe_abs_sin no_scratchpad     0              3   
                      pe_alibi   no_scratchpad     0              3   
                      pe_none    no_scratchpad     0              3   
                      pe_rotary  no_scratchpad     0              3   
                      pe_t5      no_scratchpad     0              3   
...                                                             ...   
scan--mdlen_tr25_ts48 pe_abs_sin no_scratchpad     0              3   
                      pe_alibi   no_scratchpad     0              3   
                      pe_none    no_scratchpad     0              3   
                      pe_rotary  no_scratchpad     0              3   
                      pe_t5      no_scratchpad     0              3   

                                                      is_running  is_done  \
ds                    model      scratchpad_config                          
pcfg--md_productivity pe_abs_sin no_scratchpad     0       False     True   
                      pe_alibi   no_scratchpad     0       False     True   
                      pe_none    no_scratchpad     0       False     True   
                      pe_rotary  no_scratchpad     0       False     True   
                      pe_t5      no_scratchpad     0       False     True   
...                                                          ...      ...   
scan--mdlen_tr25_ts48 pe_abs_sin no_scratchpad     0       False     True   
                      pe_alibi   no_scratchpad     0       False     True   
                      pe_none    no_scratchpad     0       False     True   
                      pe_rotary  no_scratchpad     0       False     True   
                      pe_t5      no_scratchpad     0       False     True   

                                                     launcher_id  \
ds                    model      scratchpad_config                 
pcfg--md_productivity pe_abs_sin no_scratchpad     0    3b27nkwi   
                      pe_alibi   no_scratchpad     0    27mrtr6a   
                      pe_none    no_scratchpad     0    3fknk7pc   
                      pe_rotary  no_scratchpad     0    bqoo8hoo   
                      pe_t5      no_scratchpad     0    26twvle0   
...                                                          ...   
scan--mdlen_tr25_ts48 pe_abs_sin no_scratchpad     0    1ehwwpb8   
                      pe_alibi   no_scratchpad     0    bz3encpc   
                      pe_none    no_scratchpad     0    2hkw4ihx   
                      pe_rotary  no_scratchpad     0    3m5ikkv2   
                      pe_t5      no_scratchpad     0    3ro3qgno   

                                                     compute_cluster  \
ds                    model      scratchpad_config                     
pcfg--md_productivity pe_abs_sin no_scratchpad     0       cc_narval   
                      pe_alibi   no_scratchpad     0       cc_narval   
                      pe_none  

In [38]:
xdf[(xdf["is_done"] == False) & (xdf["is_running"] == False)]

num_completed  \
ds                          model      scratchpad_config                    
s2s_addition--len_tr8_ts16  pe_t5      i0_c1_o0_v0_r0    0              0   
s2s_lego--len_tr8_ts16      pe_alibi   i0_c1_o1_v0_r1    0              2   
                            pe_none    i1_c1_o0_v0_r1    0              2   
                                       i1_c1_o1_v0_r1    0              1   
                            pe_t5      i1_c1_o1_v0_r1    0              1   
s2s_parity--len_tr8_ts16    pe_abs_sin i0_c1_o1_v1_r1    0              2   
s2s_sort--len_mltd_tr8_ts16 pe_none    no_scratchpad     0              2   
s2s_sum--len_tr8_ts16       pe_abs_sin i0_c1_o1_v1_r1    0              2   
                                       i1_c1_o0_v1_r1    0              2   
                                       i1_c1_o1_v0_r1    0              2   
                                       i1_c1_o1_v1_r1    0              2   
                            pe_alibi   i0_c1_o1_v1_r1    0              2   
                                       i1_c1_o0_v1_r1    0              2   
                                       i1_c1_o1_v0_r1    0              2   
                                       i1_c1_o1_v1_r0    0              2   
                                       i1_c1_o1_v1_r1    0              2   
                            pe_none    i0_c1_o1_v1_r1    0              2   
                                       i1_c0_o1_v1_r1    0              2   
                                       i1_c1_o0_v1_r1    0              2   
                                       i1_c1_o1_v0_r1    0              2   
                                       i1_c1_o1_v1_r0    0              2   
                                       i1_c1_o1_v1_r1    0              2   
                            pe_rotary  i0_c1_o1_v1_r1    0              1   
                                       i1_c0_o1_v1_r1    0              1   
                                       i1_c1_o0_v1_r1    0              1   
                                       i1_c1_o1_v0_r1    0              1   
                                       i1_c1_o1_v1_r0    0              1   
                                       i1_c1_o1_v1_r1    0              1   
                            pe_t5      i0_c1_o1_v1_r1    0              2   
                                       i1_c0_o1_v1_r1    0              2   
                                       i1_c1_o0_v1_r1    0              2   
                                       i1_c1_o1_v1_r0    0              2   
                                       i1_c1_o1_v1_r1    0              2   

                                                            num_seed_runs  \
ds                          model      scratchpad_config                    
s2s_addition--len_tr8_ts16  pe_t5      i0_c1_o0_v0_r0    0              0   
s2s_lego--len_tr8_ts16      pe_alibi   i0_c1_o1_v0_r1    0              3   
                            pe_none    i1_c1_o0_v0_r1    0              3   
                                       i1_c1_o1_v0_r1    0              2   
                            pe_t5      i1_c1_o1_v0_r1    0              2   
s2s_parity--len_tr8_ts16    pe_abs_sin i0_c1_o1_v1_r1    0              3   
s2s_sort--len_mltd_tr8_ts16 pe_none    no_scratchpad     0              3   
s2s_sum--len_tr8_ts16       pe_abs_sin i0_c1_o1_v1_r1    0              3   
                                       i1_c1_o0_v1_r1    0              3   
                                       i1_c1_o1_v0_r1    0              3   
                                       i1_c1_o1_v1_r1    0              2   
                            pe_alibi   i0_c1_o1_v1_r1    0              3   
                                       i1_c1_o0_v1_r1    0              3   
                                       i1_c1_o1_v0_r1    0              3   
                                       i1_c1_o1_v1_r0    0              3   
                                       i1_c1_o1_v1_r1  

In [17]:
check_runs_generated_by_launcher_id("27k8r9tj")

sw_c19ab1768f8b3039a699fab2ff637c86__146317 crashed best_run_seed_exp SW-t5_dec_base_pe_alibi_w_scratchpad_f_ufs__i1_c1_o1_v0_r1_s2s_lego_sweep___data-s2s_lego-len_tr8_ts16 ['launched_by_27k8r9tj', 'manual_sweep', 'scratchpad', 'scratchpad_f', 'scratchpad_s2s_lego', 'sweep']
sw_c19ab1768f8b3039a699fab2ff637c86__146317 SW-t5_dec_base_pe_alibi_w_scratchpad_f_ufs__i1_c1_o1_v0_r1_s2s_lego_sweep___data-s2s_lego-len_tr8_ts16
no eval results
False

sw_c19ab1768f8b3039a699fab2ff637c86__234054 finished best_run_seed_exp SW-t5_dec_base_pe_alibi_w_scratchpad_f_ufs__i1_c1_o1_v0_r1_s2s_lego_sweep___data-s2s_lego-len_tr8_ts16 ['launched_by_27k8r9tj', 'manual_sweep', 'scratchpad', 'scratchpad_f', 'scratchpad_s2s_lego', 'sweep']
True

sw_c19ab1768f8b3039a699fab2ff637c86__256788 finished best_run_seed_exp SW-t5_dec_base_pe_alibi_w_scratchpad_f_ufs__i1_c1_o1_v0_r1_s2s_lego_sweep___data-s2s_lego-len_tr8_ts16 ['launched_by_27k8r9tj', 'manual_sweep', 'scratchpad', 'scratchpad_f', 'scratchpad_s2s_lego', 'sw

In [ ]:
xxdf = xdf.reset_index()
xxdf[(xxdf["is_done"] == False) & (xxdf["ds"] == "s2s_addition--len_tr8_ts16")]

In [ ]:
xdf[(xdf["is_done"] == False) & (xdf["is_running"] == False) & (xdf["ds"] == "s2s_addition--len_tr8_ts16")]

In [ ]:
xdf.columns

## Fix Runs without summary

In [29]:
# completed_runs = set(df[(df["is_complete"] == True) & (df["job_type"] == "best_run_seed_exp")]["id"].tolist())

completed_runs = set(df[(df["job_type"] == "best_run_seed_exp")]["id"].tolist())


In [30]:
c = 0
for run in tqdm(runs):
    # if run.id not in completed_runs:
    #     continue
    if run.job_type == "agent":
        continue

    # keys_not_present = []
    # for k in target_summary_keys:
    #     if k not in run.summary:
    #         keys_not_present.append(k)
    if "pred/test_acc_overall" in run.summary:
        continue

    print(run.group)
    print(run.id)
    print(run.url)
    print("------------------")

    c += 1
    # continue

    h_all = list(run.scan_history())

    s = {}
    for h in h_all:
        for k,v in h.items():
            if k.startswith("pred/"):
                s[k] = v

    run.summary.update(s)
    run.save()

c

  0%|          | 0/1682 [00:00<?, ?it/s]

SW-t5_dec_base_pe_alibi_w_scratchpad_f_ufs__i1_c1_o0_v1_r1_s2s_sum_sweep___data-s2s_sum-len_tr8_ts16
sw_ff9488bdbc182b91baa9c78cf67258e0__146317
https://wandb.ai/kzmnjd/len_gen/runs/sw_ff9488bdbc182b91baa9c78cf67258e0__146317
------------------


  0%|          | 3/1682 [00:01<09:26,  2.97it/s]

SW-t5_dec_base_pe_rotary_w_scratchpad_f_ufs__i0_c1_o1_v1_r1_s2s_sum_sweep___data-s2s_sum-len_tr8_ts16
sw_9a7124fb2298e6bb8723f38eb6f2fbfe__234054
https://wandb.ai/kzmnjd/len_gen/runs/sw_9a7124fb2298e6bb8723f38eb6f2fbfe__234054
------------------


  0%|          | 4/1682 [00:02<15:19,  1.83it/s]

...

SW-t5_dec_base_pe_none_w_scratchpad_f_ufs__i1_c1_o0_v0_r1_s2s_lego_sweep___data-s2s_lego-len_tr8_ts16
sw_a06c80d5a23c38b743f34c33b1c6874a__146317
https://wandb.ai/kzmnjd/len_gen/runs/sw_a06c80d5a23c38b743f34c33b1c6874a__146317
------------------


 29%|██▉       | 494/1682 [00:47<00:40, 29.69it/s]

SW-t5_dec_base_pe_t5_w_scratchpad_f_ufs__i1_c1_o1_v0_r1_s2s_lego_sweep___data-s2s_lego-len_tr8_ts16
sw_98854403c39bd43c75450b7d733135a4__234054
https://wandb.ai/kzmnjd/len_gen/runs/sw_98854403c39bd43c75450b7d733135a4__234054
------------------


100%|██████████| 1682/1682 [00:48<00:00, 34.56it/s]


37